# Размещение баннеров

## Постановка задачи
Представим, что международное круизное агентство "Carnival Cruise Line" решило себя разрекламировать с помощью баннеров и обратилось для этого к Вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с Вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры - легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях - Foursqare.
Часть открытых данных есть, например, на сайте archive.org:
https://archive.org/details/201309_foursquare_dataset_umn

Скачаем любым удобным образом архив с этой страницы.

## Преобразование данных
Нас будет интересовать файл checkins.dat. Открыв его, увидим следующую структуру:

id | user_id | venue_id | latitude | longitude | created_at

---------+---------+----------+-------------------+-------------------+---------------------

984301 | 2041916 | 5222 | | | 2012-04-21 17:39:01

984222 | 15824 | 5222 | 38.8951118 | -77.0363658 | 2012-04-21 17:43:47

984315 | 1764391 | 5222 | | | 2012-04-21 17:37:18

984234 | 44652 | 5222 | 33.800745 | -84.41052 | 2012-04-21 17:43:43

...

Для удобной работы с этим документом преобразуем его к формату csv, удалив строки не содержащие координат - они неинформативны для нас:

id,user_id,venue_id,latitude,longitude,created_at

984222,15824,5222,38.8951118,-77.0363658,2012-04-21T17:43:47

984234,44652,5222,33.800745,-84.41052,2012-04-21T17:43:43

984291,105054,5222,45.5234515,-122.6762071,2012-04-21T17:39:22

...

С помощью pandas построим DataFrame и убедимся, что все 396632 строк с координатами считаны успешно.

In [16]:
import pandas as pd

In [14]:
lines = []
data = []

with open('checkins.dat') as input_file:
    columns = map(lambda x: x.strip(), input_file.readline().strip().split('|'))
    n_params = len(columns)
    for line in input_file:
        items = map(lambda x: x.strip(), line.strip().split('|'))
        if len(items) != n_params:
            print line
        else:
            data.append(items)

---------+---------+----------+-------------------+-------------------+---------------------

(1021966 rows)





In [116]:
df = pd.DataFrame(data)
df.columns = columns
df = df[(df.latitude != '') & (df.longitude != '')]
df.head()

ValueError: Length mismatch: Expected axis has 2 elements, new values have 6 elements

In [115]:
df.to_csv('data.csv')

In [95]:
df.shape

(396634, 6)

# Кластеризация
Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача - хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в [sklearn user guide](http://scikit-learn.org/stable/modules/clustering.html#mean-shift), а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

Примечание: на 396634 строках, кластеризация будет работать долго (порядка часа). Для получения корректного ответа достаточно и 100000 (~2 минуты на "среднем" ноутбуке). Быть очень терпеливым не возбраняется - результат от этого только улучшится.

In [50]:
from sklearn.cluster import MeanShift

In [51]:
data = df[['latitude', 'longitude']]
data = data.applymap(float)

In [96]:
data_sample = data.sample(100000)

In [97]:
%%time
clst = MeanShift(bandwidth = 0.1)
clst.fit(data_sample)

CPU times: user 3min 3s, sys: 3.41 s, total: 3min 7s
Wall time: 3min 14s


Некоторые из получившихся кластеров содержат слишком мало точек - такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

При желании увидеть получившиеся результаты на карте, можно передать центры получившихся кластеров в один из инструментов визуализации. Например, сайт mapcustomizer.com имеет функцию Bulk Entry, куда можно вставить центры полученных кластеров в формате:

38.8951118,-77.0363658

33.800745,-84.41052

45.5234515,-122.6762071

...

In [98]:
data_sample['cluster'] = clst.predict(data_sample)

In [117]:
cluster_size = pd.DataFrame(data_sample.pivot_table(index = 'cluster', aggfunc = 'count', values = 'latitude'))
cluster_size.columns = ['clust_size']

In [118]:
cluster_centers_df = pd.DataFrame(clst.cluster_centers_)
cluster_centers_df.columns = ['cent_latitude', 'cent_longitude']

In [119]:
cluster_df = cluster_centers_df.join(cluster_size)
cluster_df.to_csv('clusters.csv', index = None)
cluster_df = cluster_df[cluster_df.clust_size > 15]
cluster_df.head()

,cent_latitude,cent_longitude,clust_size
0,40.717516,-73.989309,14133
1,41.878190,-87.629843,3814
2,33.447277,-111.902612,2341
3,38.886986,-77.048567,2744
4,33.448474,-112.074079,2735


Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу "Carnival Cruise Line" адреса офисов:

 * 33.751277, -118.188740 (_Los Angeles_)
 * 25.867736, -80.324116 (_Miami_)
 * 51.503016, -0.075479 (_London_)
 * 52.378894, 4.885084 (_Amsterdam_)
 * 39.366487, 117.036146 (_Beijing_)
 * -33.868457, 151.205134 (_Sydney_)

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

Примечание: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико.

Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании - широта и долгота этого центра, записанные через пробел.

In [102]:
def get_distance(lat1, lon1, lat2, lon2):
    return ((lat1 - lat2)**2 + (lon1 - lon2)**2) ** 0.5

office_coordinates = [
    (33.751277, -118.188740),
    (25.867736, -80.324116),
    (51.503016, -0.075479),
    (52.378894, 4.885084),
    (39.366487, 117.036146),
    (-33.868457, 151.205134)
]

def get_min_distance_to_office(lat, lon):
    min_dist = None
    for (of_lat, of_lon) in office_coordinates:
        dist = get_distance(lat, lon, of_lat, of_lon)
        if (min_dist is None) or (dist < min_dist):
            min_dist = dist
    return min_dist

In [103]:
cluster_df['min_distance'] = map(get_min_distance_to_office, cluster_df.cent_latitude, cluster_df.cent_longitude)

In [104]:
cluster_df.sort('min_distance')[:20]

/Users/miptgirl/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,cent_latitude,cent_longitude,clust_size,min_distance
239,-33.860483,151.209228,58,0.008964
377,52.372970,4.892311,33,0.009345
54,51.502724,-0.128090,285,0.052612
48,33.811294,-118.145453,277,0.073998
272,25.907583,-80.258669,73,0.076623
23,25.787468,-80.215137,703,0.135349
166,33.896401,-118.220429,66,0.148543
83,33.901553,-118.063269,151,0.195770
107,26.036990,-80.212908,104,0.202519
37,33.921958,-118.413539,395,0.282253


In [114]:
cluster_df.to_csv('clusters.csv', index = None)